# Weight Bins Simulation

We want to simulate what would be the saving of having a larger fleet of cars. So that the 

### Imports

In [74]:
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.append('c:\\Users\\Jordi Cremades\\Documents\\Repository')

from Utilities import query_engines

### Queries

In [75]:
# Query Delta Time
query = query_engines.QueryEngines(
    query='weight_bins.sql', 
    del1='--v1', 
    del2='--endv1',
    params=None,
    load_from_output_file='weight_bins',
    output_file='weight_bins',
    printq=None
)

df = query.query_run_starbust()
df.to_csv('raw.csv')

In [76]:
# Clean dataset
columns_to_keep = [
    'country_code'
    ,'transport'
    ,'weight_tiers'
    ,'total_orders'
    ,'extra_courier_cost'
    ,'total_split_cost'
    ,'total_remake_cost'
    ,'original_big_order_agent_contacts'
    ,'extra_big_order_agent_contacts'
]
df_clean = df[columns_to_keep].copy()
df_clean['original_big_order_agent_contacts'] = df_clean['original_big_order_agent_contacts'].astype(float) # to float
# Fillna
df_clean['transport'].fillna('Undefined',inplace=True)
df_clean['extra_courier_cost'].fillna(0,inplace=True)
df_clean['total_split_cost'].fillna(0,inplace=True)
df_clean['total_remake_cost'].fillna(0,inplace=True)
df_clean['extra_big_order_agent_contacts'].fillna(0,inplace=True)
df_clean

,country_code,transport,weight_tiers,total_orders,extra_courier_cost,total_split_cost,total_remake_cost,original_big_order_agent_contacts,extra_big_order_agent_contacts
0,TN,Undefined,"[26000.0,27000.0)",2,1.042771,0.000,1.042771,1.0,0.0
1,KG,MOTORBIKE,"[3000.0,4000.0)",477,1.186338,0.000,1.186338,1.0,0.0
2,ES,BICYCLE,Undefined,133792,1221.582200,19.852,1198.962200,430.0,24.0
3,ES,WALKER,"[20000.0,21000.0)",28,1.716000,0.000,1.716000,0.0,0.0
4,MA,Undefined,"[25000.0,26000.0)",72,16.120560,0.000,16.120560,14.0,9.0
...,...,...,...,...,...,...,...,...,...
3299,AM,MOTORBIKE,"[9000.0,10000.0)",98,0.000000,0.000,0.000000,10.0,0.0
3300,TN,MOTORBIKE,"[2000.0,3000.0)",1104,6.014831,0.000,6.014831,5.0,0.0
3301,AM,MOTORBIKE,"[12000.0,13000.0)",36,0.000000,0.000,0.000000,4.0,0.0
3302,HR,MOTORBIKE,"[37000.0,38000.0)",3,4.540000,4.540,0.000000,0.0,0.0


In [78]:
# Check that Extra Courier Cost = Total Split Cost + Total Remake Cost

# Compute the difference
df_clean['extra_courier_diff'] = df_clean['extra_courier_cost'] - df_clean['total_split_cost'] - df_clean['total_remake_cost']
perc_difference = 100*df_clean['extra_courier_diff'].sum()/df_clean['extra_courier_cost'].sum()
print('Extra courier cost value and computation difference is {:.2f}%'.format(perc_difference))

# The difference is not significant, we can use the total_extra_courier_cost column for our purposes

Extra courier cost value and computation difference is 1.13%


In [73]:
# 1) Compute the extra courier cost form CARS and NO CARS and provide the difference in each segment

# First component of the cost --> Extra courier cost
df_clean['final_cost_1'] = df_clean['extra_courier_cost']

# Second component of the cost --> #Contacts x 1.05euros
df_clean['final_cost_2'] = (df_clean['original_big_order_agent_contacts'] + df_clean['extra_big_order_agent_contacts'])*1.05
df_clean['final_cost'] = df_clean['final_cost_1'] + df_clean['final_cost_2'] 

# Compute the ratio of cost and orders
df_clean['ratio_cost_x_order'] = df_clean['final_cost']/df_clean['total_orders']

# Build the only cars df
cond_cars = df_clean['transport'] == 'CAR'
df_cars = df_clean[cond_cars].copy()
df_cars.to_csv('question2.csv')

# Join dataframes and simulate cost if all ordres had been with CARS
conclusion = pd.merge(df_clean, df_cars, on=['country_code','weight_tiers'],suffixes=('','_cars'), how='left')
conclusion.to_csv('question1.csv')

In [72]:
# Join it with the info from the no cars
df_final = pd.merge(df_clean, df_cars, on=['country_code','weight_tiers'],suffixes=('','_cars'), how='left')
df_final['diff'] = df_final['total_orders']*df_final['ratio_cars']-df_final['final_cost']
df_final.to_csv('final.csv')

In [72]:
df_cars['df_']

,country_code,transport,weight_tiers,total_orders,delivered_orders,extra_courier_cost,total_split_cost,total_remake_cost,original_big_order_agent_contacts,extra_big_order_agent_contacts,extra_courier_diff,final_cost_1,final_cost_2,final_cost,ratio_cost_x_order
9,PL,CAR,"[3000.0,4000.0)",43530,43530,20.386751,0.000000,20.386751,13.0,1.0,0.000000e+00,20.386751,14.70,35.086751,0.000806
16,MA,CAR,"[20000.0,21000.0)",21,21,10.081369,1.443065,8.638304,4.0,1.0,-1.776357e-15,10.081369,5.25,15.331369,0.730065
22,BG,CAR,"[14000.0,15000.0)",1211,1211,0.000000,0.000000,0.000000,11.0,0.0,0.000000e+00,0.000000,11.55,11.550000,0.009538
24,NG,CAR,"[4000.0,5000.0)",359,359,0.000000,0.000000,0.000000,0.0,0.0,0.000000e+00,0.000000,0.00,0.000000,0.000000
25,ME,CAR,"[23000.0,24000.0)",132,132,27.072100,0.000000,27.072100,0.0,1.0,0.000000e+00,27.072100,1.05,28.122100,0.213046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3284,KG,CAR,"[6000.0,7000.0)",167,167,0.000000,0.000000,0.000000,4.0,0.0,0.000000e+00,0.000000,4.20,4.200000,0.025150
3295,NG,CAR,"[6000.0,7000.0)",177,177,1.570164,0.000000,1.570164,3.0,0.0,0.000000e+00,1.570164,3.15,4.720164,0.026668
3297,ME,CAR,"[33000.0,34000.0)",42,42,98.350900,54.185300,44.165600,3.0,0.0,2.131628e-14,98.350900,3.15,101.500900,2.416688
3298,ME,CAR,"[28000.0,29000.0)",84,84,154.565500,78.549100,76.016400,0.0,0.0,1.421085e-14,154.565500,0.00,154.565500,1.840065
